In [11]:
# Import packages needed & load image
import gc
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import platform


from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pandas as pd

import general_funcs

# OS related settings
if platform.system() == 'Windows':
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
    # %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    ;
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



# image_file = askopenfilename(title='Load image file', initialdir='./data/field_image')
# image_file_ref = askopenfilename(title='Load image file', initialdir='./data/field_image')
# mtp_file = askopenfilename(title='Load mtp file', initialdir='./data/mtp')
# mtp_file_ref = askopenfilename(title='Load mtp file', initialdir='./data/mtp')

image_file = "/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/field_image/BRC/BRC_20190904_121517_Thermal.tif"
image_ref_file = "/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/field_image/BRC/BRC_20190814_141033_RGB.tif"
mtp_file = "/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/mtp/BRC_20190904_121517_Thermal_mtp.pkl"
mtp_ref_file = "/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/mtp/BRC_20190814_141033_RGB_mtp.pkl"


try:
    with open(mtp_file, 'rb') as f:
        mtp = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))

try:
    with open(mtp_ref_file, 'rb') as f:
        mtp_ref = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))

ds_ref = gdal.Open(image_ref_file)
gt_ref = ds_ref.GetGeoTransform()
mtp_ref_geo = []

for point in mtp_ref:
    geo_loc = general_funcs.pix2geo(point, gt_ref)
    mtp_ref_geo.append(geo_loc)
    

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
mtp_geo = []
for point in mtp:
    geo_loc = general_funcs.pix2geo(point, gt)
    mtp_geo.append(geo_loc)
    
mtp_ref_geo = np.array(mtp_ref_geo)
mtp_geo = np.array(mtp_geo)

shift_geo = mtp_ref_geo - mtp_geo

# plot_loc_file_ref = askopenfilename(title='Load plot location file', initialdir='./data/plot_location')
plot_loc_file_ref = "/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/plot_location/BRC_20190814_141033_RGB_plot_loc_TEST.pkl"

try:
    with open(plot_loc_file_ref, 'rb') as f:
        interested_area_ref = pickle.load(f)
        plot_vertices_gps_ref = pickle.load(f)
        plot_notes_ref = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))
    
    
plot_vertices_ref = general_funcs.plotVGPS2plotV(plot_vertices_gps_ref, gt_ref)


plot_vertices = {}
plot_vertices_before_transform = {}
for plot_name in plot_vertices_ref.keys():
    one_plot_vertices_ref = plot_vertices_ref[plot_name]
    one_plot_vertices = []
    one_plot_vertices_before_transform = []
    for pix_loc_ref in one_plot_vertices_ref:
        geo_loc_ref = general_funcs.pix2geo(pix_loc_ref, gt_ref)
        point_shift_geo = general_funcs.find_point_in_img_ref(pix_loc_ref, mtp, mtp_ref, shift_geo)
        geo_loc = geo_loc_ref - point_shift_geo
        pix_loc = general_funcs.geo2pix(geo_loc, gt)
        pix_loc_before_transform = general_funcs.geo2pix(geo_loc_ref, gt)
        
        one_plot_vertices.append(pix_loc)
        one_plot_vertices_before_transform.append(pix_loc_before_transform)
        
    one_plot_vertices = np.array(one_plot_vertices)
    one_plot_vertices_before_transform = np.array(one_plot_vertices_before_transform)
    
    plot_vertices[plot_name] = one_plot_vertices
    plot_vertices_before_transform[plot_name] = one_plot_vertices_before_transform
    
# plot_vertices = np.array(plot_vertices)
# plot_vertices_before_transform = np.array(plot_vertices_before_transform)


img = io.imread(image_file)
# img_ref = io.imread(image_ref_file)
layer_RGB, layer_IR, layer_mask = general_funcs.extract_layers(img)
# layer_RGB_ref, layer_IR_ref, layer_mask_ref = general_funcs.extract_layers(img_ref)

h, w, d = img.shape
# h_ref, w_ref, d_ref = img_ref.shape



Interactive plot activated


In [12]:
fig, ax = plt.subplots(figsize=(7, 7))
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
if d == 5 or d == 4:
    myax = ax.imshow(layer_RGB)
elif d == 2:
    mask_not_0_inds = np.where(layer_mask > 0)
    vmin, vmax = general_funcs.cal_vmin_vmax(layer_IR, layer_mask)
    myax = ax.imshow(layer_IR, cmap='gist_gray', vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(myax)
    
for plot_name in plot_vertices.keys():
    one_plot_vertices = plot_vertices[plot_name]
    polygon = patches.Polygon(one_plot_vertices, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
    ax.add_patch(polygon)
    text_loc = np.mean(one_plot_vertices, 0)
    axtx = ax.text(text_loc[0], text_loc[1], plot_name, ha='center', va='center')